# Export Album Data from MusicBrainz Database

Some imports first:

In [1]:
import json
import os
import re
import warnings

import PyLyrics

from sqlalchemy import (create_engine, MetaData, Table, Column, 
                        ForeignKey, select, func, Integer, text,
                        String, distinct, bindparam)
from sqlalchemy import exc as sa_exc
from sqlalchemy.dialects.postgresql import ARRAY
from sqlalchemy.ext import automap
from sqlalchemy.orm import sessionmaker, class_mapper
from sqlalchemy.sql.expression import label, case
from sqlalchemy.sql import literal_column


And some global settings:

In [2]:
db_url = 'postgresql+psycopg2://musicbrainz@localhost:6543/musicbrainz'

dump_filename = 'musicbrainz.json'
lyrics_dir = 'lyrics'
final_filename = 'musicbrainz-with-lyrics.json'

### Connect to the local database

In [3]:
engine = create_engine(db_url)
conn = engine.connect()
Session = sessionmaker(bind=engine)
session = Session()

### Reflect the tables we need

The two schemes used are _musicbrainz_ which is the default scheme, and _cover_art_archive_.


In [4]:
tables = """release_group release_group_primary_type release 
release_status release_country release_unknown_country release_packaging
release_label label label_type artist artist_credit artist_credit_name 
artist_type gender area medium track country_area area""".split()

ca_tables = "cover_art cover_art_type".split()

with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=sa_exc.SAWarning)

    meta = MetaData()
    meta.reflect(bind=engine, only=tables)
    Base = automap.automap_base(metadata=meta)
    Base.prepare()
    
    ca_meta = MetaData(schema='cover_art_archive')
    ca_meta.reflect(bind=engine, only=ca_tables) 
    CABase = automap.automap_base(metadata=ca_meta)
    CABase.prepare()

# only do this when it is ok to be lazy!
loc = locals()

camelize = lambda s: s[0].upper() + \
    re.sub(r'_([a-z])', lambda m: m.group(1).upper(), s[1:])

for table in tables:
    loc[camelize(table)] = getattr(Base.classes, table)
    
for table in ca_tables:
    loc[camelize(table)] = getattr(CABase.classes, table)

### Now, let's have a look at the MusicBrainz database:

Hint:
* The _release_group_ is the abstract Album (exampel Jimi Hendrix - Electric Ladyland)
* A _release_ is a physical (or digital, nowadays) Album with a bar code, maybe some country specific songs on it, etc.

![Musicbrainz Schema](https://wiki.musicbrainz.org/-/images/5/52/ngs.png "Title")

### Two subqueries

First we find the original release date/year by creation a union table of _release_country_ and _release_unknown_country_:

In [5]:
# Union of ReleaseCountry and ReleaseUnknownCountry
q1 = session.query(
    label('release', ReleaseCountry.release), 
    label('date_year', ReleaseCountry.date_year),
    label('date', case([(ReleaseCountry.date_year == None, None)], else_=func.concat(
        ReleaseCountry.date_year, '-', 
        text("COALESCE(LPAD(date_month::text, 2, '0'), '12')"), '-', 
        text("COALESCE(LPAD(date_day::text, 2, '0'), '12')"))
    )),
)

q2 = session.query(
    label('release', ReleaseUnknownCountry.release),
    label('date_year', ReleaseUnknownCountry.date_year),
    label('date', case([(ReleaseUnknownCountry.date_year == None, None)], else_=func.concat(
        ReleaseUnknownCountry.date_year, '-', 
        text("COALESCE(LPAD(date_month::text, 2, '0'), '12')"), '-', 
        text("COALESCE(LPAD(date_day::text, 2, '0'), '12')"))
    )),
)

ReleaseDate = q1.union_all(q2).subquery()

As for the packaging, we want an array containing all the different packaging for a given release_group.

In [6]:
# Aggregate all packagings available in a release_group
Packaging = session.query(
    Release.release_group.label('release_group'),
    func.array_agg(distinct(ReleasePackaging.name), type_=ARRAY(String, as_tuple=True))\
        .label('packaging'),
   
)
Packaging = Packaging.join(ReleasePackaging, Release.packaging == ReleasePackaging.id)
Packaging = Packaging.group_by(Release.release_group)
Packaging = Packaging.subquery()

### The main query

First the list with all the values we want to fetch from the database.

In [7]:
q = session.query(
        Release.id, 
        Release.gid, 
        Release.name, 
        Release.release_group, 
        ReleaseGroupPrimaryType.name,
        ReleaseDate.c.date_year,
        ReleaseDate.c.date, 
        ArtistCredit.name, 
        Artist.name, 
        Artist.begin_date_year,
        Artist.end_date_year,
        Gender.name,
        ArtistType.name,
        Packaging.c.packaging,
        Label.name
)

Now, there are a lot of tables to join. Some are inner, some outer joins:

In [8]:
q = q.join(ReleaseGroup, Release.release_group == ReleaseGroup.id)
q = q.outerjoin(ReleaseGroupPrimaryType, ReleaseGroupPrimaryType.id == ReleaseGroup.type)
q = q.join(ArtistCredit, Release.artist_credit == ArtistCredit.id)
q = q.join(ArtistCreditName, ArtistCredit.id == ArtistCreditName.artist_credit)
q = q.join(Artist, ArtistCreditName.artist == Artist.id)
q = q.outerjoin(Gender, Artist.gender == Gender.id)
q = q.outerjoin(ArtistType, Artist.type == ArtistType.id)
q = q.outerjoin(ReleaseLabel, ReleaseLabel.release == Release.id)
q = q.outerjoin(Label, ReleaseLabel.label == Label.id)
q = q.join(Medium, Medium.release == Release.id)
q = q.join(CoverArt, CoverArt.release == Release.id)
q = q.join(CoverArtType, CoverArtType.id == CoverArt.id)

# Also join the results from the subqueries
q = q.outerjoin(ReleaseDate, ReleaseDate.c.release == Release.id)
q = q.outerjoin(Packaging, Packaging.c.release_group == Release.release_group)

Find everything we are interested in:

In [9]:
q = q.filter(Release.language == 145, Release.script == 28) # German with latin script
q = q.filter(Release.status == 1) # Official release
q = q.filter(ArtistCreditName.position == 0)   # Additional info only for main Artist
q = q.filter(Artist.id != 1) # No various artists releases
q = q.filter(CoverArtType.type_id == 1)  # only albums that have a front cover image
q = q.filter(Medium.position == 1)   # filter some weird albums

Finally, we are only interested in one single entry per _release_group_. On PostgreSQL this can be done with a combination of a <code>DISTINCT ON &lt;COLUMN></code> and <code>ORDER BY</code> clause. In our case, we arbitrarily prefer CD releases with the lowest number of tracks.

In [10]:
q = q.distinct(Release.release_group)
q = q.order_by(Release.release_group, Medium.format, Medium.track_count)

Let's see, with how many albums we are dealing with:

In [11]:
#q = q.filter(ArtistCredit.name == 'Die Ärzte')#, Release.name == 'Planet Punk')

total_releases = q.count()
total_releases

6404

Hm, quite a bunch.

### Additional queries 
While going through all the albums, we need two additional queries to fetch the tracks and a list with all the featuring artists:

In [12]:
track_q = session.query(Track, ArtistCredit.name)
track_q = track_q.join(ArtistCredit, Track.artist_credit == ArtistCredit.id)
track_q = track_q.join(Medium, Track.medium == Medium.id)
track_q = track_q.filter(Medium.release == bindparam('release'))
track_q = track_q.order_by(Medium.position, Track.position)

feat_q = session.query(ArtistCreditName.name).distinct()
feat_q = feat_q.join(ArtistCredit, ArtistCredit.id == ArtistCreditName.artist_credit)
feat_q = feat_q.filter(ArtistCredit.artist_count > 1, ArtistCreditName.position > 1)
feat_q = feat_q.join(Track, Track.artist_credit == ArtistCredit.id)
feat_q = feat_q.join(Medium, Medium.id == Track.medium)
feat_q = feat_q.filter(Medium.release == bindparam('release'))

### Assemble the JSON and write it to a file

This takes some time.

In [14]:
data = []
i = 0

for (release_id, release_gid, release_name, release_group, release_type, 
     release_year, release_date, credit_name, artist_name, 
     artist_begin_year,  artist_end_year, artist_gender, artist_type, 
     packaging, label_name) in q:
    
    i += 1
    if (i%500 == 0): print(i, '/', total_releases)

    tracks = track_q.params(release=release_id)
    track_list = [
        {
            "gid": track.gid,
            "number": track.number,
            "title": track.name,
            "artist": artist_credit,
            "length": track.length // 1000 if track.length else 0,
        } for track, artist_credit in tracks
    ]
    
    feats = [feat for feat in feat_q.params(release=release_id)]
    
    data.append({
        "gid": release_gid,
        "album_title" : release_name,
        "album_release_year": release_year,
        "album_release_date": release_date,
        "album_artist": credit_name,
        "main_artist": {
            "name": artist_name, 
            "gender": artist_gender,
            "artist_type": artist_type,
            "active_from": artist_begin_year,
            "active_until": artist_end_year,
        },
        "cover_image": "http://coverartarchive.org/release/{}/front".format(release_gid),
        "packaging": list(packaging) if packaging else [],
        "label": label_name,
        "release_type": release_type,
        "featurings": feats,
        "track_count": len(track_list),
        "tracks": track_list,
    })

json.dump(data, open(dump_filename, 'w'), sort_keys=True, 
          indent=4, separators=(',', ': '))

500 / 6404
1000 / 6404
1500 / 6404
2000 / 6404
2500 / 6404
3000 / 6404
3500 / 6404
4000 / 6404
4500 / 6404
5000 / 6404
5500 / 6404
6000 / 6404


#### Let's see the result

In [15]:
print(''.join(open(dump_filename).readlines()[:30]))
print('...')

[
    {
        "album_artist": "In Extremo",
        "album_release_date": "1999-08-30",
        "album_release_year": 1999,
        "album_title": "Verehrt und angespien",
        "cover_image": "http://coverartarchive.org/release/3ebfa0e4-0f05-4719-a8ed-d4bee5683e61/front",
        "featurings": [],
        "gid": "3ebfa0e4-0f05-4719-a8ed-d4bee5683e61",
        "label": "Mercury Records",
        "main_artist": {
            "active_from": 1995,
            "active_until": null,
            "artist_type": "Group",
            "gender": null,
            "name": "In Extremo"
        },
        "packaging": [
            "Digipak",
            "Jewel Case"
        ],
        "release_type": "Album",
        "track_count": 13,
        "tracks": [
            {
                "artist": "In Extremo",
                "gid": "7b6fb6c0-9431-3073-86a1-ccc56428cda9",
                "length": 268,
                "number": "1",
                "title": "Merseburger Zauberspr\u00fcche"

...


## Fetch Lyrics with PyLyrics

In [ ]:
for i, album in enumerate(json.load(open(dump_filename))):
    if (i % 10 == 0): print(i, '/', total_releases)
    path = os.path.join(lyrics_dir, album['gid'])
    if not os.path.isdir(path):
        os.mkdir(path)
    for track in album['tracks']:
        lyrics_file = os.path.join(path, track['gid'])
        if not os.path.exists(lyrics_file):
            try:
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    lyrics = PyLyrics.PyLyrics.getLyrics(track['artist'], track['title'])
            except ValueError:
                lyrics = ''
            open(lyrics_file, 'w').write(lyrics)

0 / 6404
10 / 6404
20 / 6404
30 / 6404
40 / 6404
50 / 6404
60 / 6404
70 / 6404
80 / 6404
90 / 6404
100 / 6404
110 / 6404
120 / 6404
130 / 6404
140 / 6404
150 / 6404
160 / 6404
170 / 6404
180 / 6404
190 / 6404
200 / 6404
210 / 6404
220 / 6404
230 / 6404
240 / 6404
250 / 6404
260 / 6404
270 / 6404
280 / 6404
290 / 6404
300 / 6404
310 / 6404
320 / 6404
330 / 6404
340 / 6404
350 / 6404
360 / 6404
370 / 6404
380 / 6404
390 / 6404
400 / 6404
410 / 6404
420 / 6404
430 / 6404
440 / 6404
450 / 6404
460 / 6404
470 / 6404
480 / 6404
490 / 6404
500 / 6404
510 / 6404
520 / 6404
530 / 6404
540 / 6404
550 / 6404
560 / 6404
570 / 6404
580 / 6404
590 / 6404
600 / 6404
610 / 6404
620 / 6404
630 / 6404
640 / 6404
650 / 6404
660 / 6404
670 / 6404
680 / 6404
690 / 6404
700 / 6404
710 / 6404
720 / 6404
730 / 6404
740 / 6404
750 / 6404
760 / 6404
770 / 6404


## Insert Lyrics in to JSON

In [5]:
data = json.load(open(dump_filename))
    
for i, album in enumerate(data):
    if album:
        dir_path = os.path.join(lyrics_dir, album['gid'])
        if os.path.isdir(dir_path):
            for track in album['tracks']:
                file_path = os.path.join(dir_path, track['gid'])
                if os.path.exists(file_path):
                    with open(file_path) as lf:
                        track['lyrics'] = lf.read()
        
with open(final_filename, 'w') as out_file:
    json.dump(data, out_file, sort_keys=True, indent=4, separators=(',', ': '))